# Megamaster

In [1]:
import pandas as pd
import re
from pathlib import Path
import os
import numpy as np

pd.set_option('display.max_columns', 500)

In [2]:
from ipyfilechooser import FileChooser

fc = FileChooser("C:/Playground/bhaiya-orders/zoho-bhor-farms/inputs/")
fc.use_dir_icons = True
display(fc)

FileChooser(path='C:/Playground/bhaiya-orders/zoho-bhor-farms/inputs/', filename='', show_hidden='False')

In [3]:
FILE = Path(fc.selected)

In [4]:
OUTPUT_DIR = Path("C:/Playground/bhaiya-orders/zoho-bhor-farms/outputs/")

save_pth = Path(OUTPUT_DIR/str(FILE.stem)); save_pth
os.makedirs(save_pth, exist_ok=True)

PDF_DIR = save_pth/'customerwise-pdfs'
os.makedirs(PDF_DIR, exist_ok=True)

In [5]:
# items2subset = ['Gawar/Cluster Beans', 'Mint Leaves', 'Sponge Gourd',  'Tondli', 'VNR Guava', ]
items2subset = None
STATUSES2PROCESS = ['Completed', 'Network Error']

In [6]:
try:
    if 'csv' in FILE.suffix: df = pd.read_csv(FILE)
    else: 
        df = pd.read_excel(FILE)
        if len(pd.ExcelFile(FILE).sheet_names) > 1:
            print("WARNING: MORE THAN ONE SHEETS PRESENT IN FILE")
            
except OSError:
    print("Error occured, invalid file!")

In [7]:
df.head(3)

,Name,Select Drop Point,Enter Building Dosti,Select Building Ivory,Select Building Wellness Forever,Select Building NCP,Select Building Ajmera iLand,Select Building Sector 1/2,Select Building Cosmic,Select Building R2,Select Building R12,Select Building R1,Select Wing,Flat No,Phone,Email,Select Delivery Date,Opt in for Doorstep Delivery @ Rs 25,Potatoes,Potatoes Price,Select Quantity of Potatoes,Red Onions,Red Onions Price,Select Quantity of Red Onions,Cauliflower,Cauliflower Price,Select Quantity of Cauliflower,Cabbage,Cabbage Price,Select Quantity of Cabbage,Brinjal Bharta,Brinjal Bharta Price,Select Quantity of Brinjal Bharta,Pumpkin,Pumpkin Price,Select Quantity of Pumpkin,Brinjal Kateri,Brinjal Kateri Price,Select Quantity of Brinjal Kateri,Cucumber,Cucumber Price,Select Quantity of Cucumber,Bhindi/Lady's Finger,Bhindi/Lady's Finger Price,Select Quantity of Bhindi/Lady's Finger,Bitter Gourd,Bitter Gourd Price,Select Quantity of Bitter Gourd,Ridge Gourd,Ridge Gourd Price,Select Quantity of Ridge Gourd,Beetroot,Beetroot Price,Select Quantity of Beetroot,Button Mushrooms,Button Mushroom Price,Select Quantity of Button Mushroom,Bottle Gourd,Bottle Gourd Price,Select Quantity of Bottle Gourd,Drumsticks,Drumsticks Price,Select Quantity of Drumsticks,Capsicum,Capsicum Price,Select Quantity of Capsicum,Gawar/Cluster Beans,Gawar/Cluster Beans Price,Select Quantity of Gawar/Cluster Beans,French Beans,French Beans Price,Select Quantity of French Beans,Tomato,Tomato Price,Select Quantity of Tomato,Sweet Corn,Sweet Corn Price,Select Quantity of Sweet Corn,Broccoli,Broccoli Price,Select Quantity of Broccoli,Lettuce,Lettuce Price,Select Quantity of Lettuce,Kiwi Zespri Imported,Kiwi Zespri Imported Price,Select Quantity of Kiwi Zespri Imported,Banana Regular,Banana Price,Select Quantity of Banana,Garlic,Garlic Price,Select Quantity of Garlic,Ginger,Ginger Price,Select Quantity of Ginger,Green Chilly,Green Chilly Price,Select Quantity of Green Chilly,Lemons,Lemons Price,Select Quantity of Lemons,Curry Leaves,Curry Leaves Price,Select Quantity of Curry Leaves,Coriander Leaves,Coriander Leaves Price,Select Quantity of Coriander Leaves,Mint Leaves,Mint Leaves Price,Select Quantity of Mint Leaves,Palak/Spinach,Palak/Spinach Price,Select Quantity of Palak/Spinach,Shepu/Dill Leaves,Shepu/Dill Leaves Price,Select Quantity of Shepu/Dill Leaves,Methi Leaves,Methi Leaves Price,Select Quantity of Methi Leaves,Red/Yellow Bell Peppers,Red/Yellow Bell Peppers Price,Select Quantity of Red/Yellow Bell Peppers,Carrots,Carrots Price,Select Quantity of Carrots,Radish/Muli,Radish/Muli Price,Select Quantity of Radish/Muli,Green Peas,Green Peas Price,Select Quantity of Green Peas,Lal Math/Amaranthus Leaves,Lal Math/Amaranthus Price,Select Quantity of Lal Math/Amaranthus,Arbi Leaves,Arbi Leaves Price,Select Quantity of Arbi Leaves,Arbi/Colocasia,Arbi/Colocasia Price,Select Quantity of Arbi/Colocasia,Babycorn,Babycorn Price,Select Quantity of Babycorn,Raw Banana,Raw Banana Price,Select Quantity of Raw Banana,Lemongrass,Lemongrass Price,Select Quantity of Lemongrass,Sponge Gourd,Sponge Gourd Price,Select Quantity of Sponge Gourd,Peeled Garlic,Peeled Garlic Price,Select Quantity of Peeled Garlic,Green Zucchini,Green Zucchini Price,Select Quantity of Green Zucchini,Yellow Zucchini,Yellow Zucchini Price,Select Quantity of Yellow Zucchini,Indian Avacado,Indian Avacado Price,Select Quantity of Indian Avacado,Apple Ber,Apple Ber Price,Select Quantity of Apple Ber,Papaya,Papaya Price,Select Quantity of Papaya,Jaggery,Jaggery Price,Select Quantity of Jaggery,Desi Peru,Desi Peru Price,Select Quantity of Desi Peru,Basil Leaves,Basil Leaves Price,Select Quantity of Basil Leaves,Total Amount,Payment Amount,Payment Status,Payment Currency,Added Time,CRM Status,Referrer Name,Task Owner
0,Nisha; Jain,R2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zinnia,NaN,NaN,NaN,902,9167306223,Prof.dr.nisha@gmail.com,Thu Oct 29th,NaN,NaN,46,NaN,NaN,82,1.0,NaN,48,1.0,NaN,42,NaN,NaN,45,NaN,NaN,22,1.0,NaN,37,N

In [8]:
len(df)

79

In [9]:
# df.tail(10)

In [10]:
df['Select Delivery Date'].unique()

array(['Thu Oct 29th'], dtype=object)

In [11]:
# df = df[df['Select Delivery Date'] == 'Tue Oct 13th']

In [12]:
assert len(df['Select Delivery Date'].unique()) == 1

In [13]:
df['Select Delivery Date'].value_counts()

Thu Oct 29th    79
Name: Select Delivery Date, dtype: int64

## Add indexing key

In [14]:
df['key'] = df['Name'] + ' ' + df['Added Time'].astype(str)

## Pull out the Select Building columns

In [15]:
bldg_cols = []

for col in df.columns:
    if 'select building' in col.lower() or 'enter building' in col.lower():
        bldg_cols.append(col)

df['building'] = df[bldg_cols].fillna('').sum(axis=1)

In [16]:
# df['building'] = df['Enter Name of Building'].fillna('')

## Subset only successfully paid orders

In [17]:
df['Payment Status'].unique()

array(['Completed', 'Processing', 'Failed'], dtype=object)

In [18]:
# df = df[df['Payment Status'].isin(STATUSES2PROCESS)]

In [19]:
# verification
df['Payment Status'].unique()

array(['Completed', 'Processing', 'Failed'], dtype=object)

## Generating Tokens

Sort by Drop Point, Building, Wing and then generate Token Values.

In [20]:
df = df.sort_values(['Select Drop Point', 'building', 'Select Wing'])
# df = df.sort_values('building')
df['Token No'] = range(1, len(df)+1)

## Check for duplicate entries

In [21]:
assert len(df) == len(df['key'].unique())

## Build Items Dictionary

    orders dictionary, one record looks like:

    {
        key:
        {
            name : str,
            drop_point : str,
            building : str,
            wing : str,
            flat : str,
            phone : numeric,
            email : str,
            delivery_date : datetime,
            order : dict,
            total_amount : float,
            payment_status : str
        }
    }


    items_dict:

    {
        item_name : 
        {
            sell_price : float,
            cost_price : float,
            packing_size : // to be read from another excel
        }
    }

If 'quantity of' occurs in column header, leave the first three words and extract the rest as your item key.

In [22]:
# extremely naive, needs a fix
all_items = [" ".join(i.split()[3:]) for i in df.columns if ('quantity of' in i.lower() or 'quantity for' in i.lower())]

In [23]:
items_dict = {}

In [24]:
def get_sell_price(item):
    try:
        if 'juice' in item.lower():
            val = df['Yoga Pulp Juice Price'].values[0]
        else:
            val = df[item + ' ' + 'Price'].values[0]
        return val
    
    except Exception as e:
        print(e)
        return 1

def get_cost_price(item):
    try:
        if 'juice' in item.lower():
            val = df['Yoga Pulp Juice Cost Price'].values[0]
        else:
            val = df[item + ' ' + 'Cost Price'].values[0]
        return val
    
    except:
        return 1

In [25]:
df_psize = pd.read_csv("C:/Playground/bhaiya-orders/zoho-bhor-farms/database/Packing Size.csv")
psize_dict = df_psize.set_index('Item').to_dict(orient='index')

In [26]:
def get_packing_size(item):
    if item in psize_dict.keys():
        return psize_dict[item]['Size']
    return 1

def get_unit(item):
    if item in psize_dict.keys():
        return psize_dict[item]['Unit']
    return 1

In [27]:
for item in all_items:
    dct = {}
    dct['packing_size'] = get_packing_size(item)
    dct['unit'] = get_unit(item)
#     if item == 'Cherry Tomatoes': item = 'Cherry Tomato'
#     if item == 'Potatoes': item = 'Potato'
    
    dct['sell_price'] = get_sell_price(item)
    dct['cost_price'] = get_cost_price(item)
    items_dict[item] = dct

## Build Master Dictionary

In [28]:
# # because few flats were not captured
df['Flat No'] = df['Flat No'].fillna('NOT CAPTURED')

In [29]:
df.fillna(0, inplace=True)

In [30]:
# create a mapping from item_id to the corresponding column to make sure no item goes unmapped

item2dropdowncol = {i : '' for i in all_items}

for index, row in df.iterrows():
    for col in df.columns:
        if 'quantity of' in col.lower() or 'quantity for' in col.lower():
            item2dropdowncol[" ".join(col.split()[3:])] = col
            
assert len(item2dropdowncol) == len(all_items)
for k, v in item2dropdowncol.items():
    assert len(v) > 3


    {
        key:
        {
            name : str,
            drop_point : str,
            building : str,
            wing : str,
            flat : str,
            phone : numeric,
            email : str,
            delivery_date : datetime,
            order : dict,
            total_amount : float,
            payment_status : str
        }
    }

In [31]:
def get_order(row):
    o = {}
    for item, qty_col in item2dropdowncol.items():
#         try:
        if row[qty_col] > 0:
            o[item] = row[qty_col]
#         except:
#             i = 'hello'
            
    return o

In [32]:
def get_total_cost_price(order):
    totalcp = 0
    for item, qty in order.items():
#         if item == 'Cherry Tomatoes': item = 'Cherry Tomato'
#         if item == 'Potatoes': item = 'Potato'
        totalcp += qty * get_cost_price(item)
    return totalcp

In [33]:
orders_dict = {}

for index, row in df.iterrows():
    dct = {}
    dct['name'] = row['Name']
    dct['drop_point'] = row['Select Drop Point']
    dct['building'] = row['building']
    dct['wing'] = row['Select Wing']
    dct['flat_no'] = row['Flat No']
    dct['phone_no'] = row['Phone']
    dct['email'] = row['Email']
    dct['delivery_date'] = row['Select Delivery Date']
    dct['total_amount'] = row['Total Amount']
    dct['payment_status'] = row['Payment Status']
    dct['order'] = get_order(row)
    dct['total_cost_price'] = get_total_cost_price(dct['order'])
    dct['profit'] = dct['total_amount'] - dct['total_cost_price']
    orders_dict[row['Token No']] = dct

# Subset Items : To Process Refunds

In [54]:
# execute only if items2subset > 0

In [34]:
# sorted(items_dict.keys())

In [112]:
items2subset = [ 'Drumsticks', 'French Beans',]

In [113]:
rows_list = []

for token, o in orders_dict.items():
    for item in items2subset:
        if item in o['order'].keys():
            dct = {}
            dct['token'] = token
            dct['name'] = o['name']
            dct['phone_no'] = o['phone_no']
            dct['email'] = o['email']
            dct['item_name'] = item
            dct['item_qty'] = o['order'][item]
            dct['item_price_per_unit'] = items_dict[item]['sell_price']
            dct['item_amount'] = items_dict[item]['sell_price'] * dct['item_qty']
            dct['amount_paid'] = o['total_amount']
            dct['payment_status'] = o['payment_status']
            
            rows_list.append(dct)

In [114]:
subset_df = pd.DataFrame(rows_list)

In [115]:
subset_df.item_name.unique()

array(['Drumsticks', 'French Beans'], dtype=object)

In [116]:
amount2refund = subset_df.groupby('name').sum()['item_amount'].to_dict()

In [117]:
subset_df['amount_to_refund'] = subset_df.name.apply(lambda x: amount2refund[x])

In [118]:
fname = save_pth/('unavl-'+FILE.stem+'.xlsx')

In [119]:
with pd.ExcelWriter(fname) as writer:
    subset_df.to_excel(writer, sheet_name='itemwise', index=False)
    (subset_df.groupby(['name', 'phone_no'])
        .first()[['email', 'amount_paid', 'amount_to_refund']]
        .to_excel(writer, sheet_name='refund_amount_agg'))

# Create Listified File

In [34]:
dict_listified = {i : 0 for i in all_items}

In [35]:
for key, value in orders_dict.items():
    for item, qty in value['order'].items():
        dict_listified[item] += qty

In [36]:
df_listified = pd.DataFrame()

df_listified['item'] = all_items
df_listified['packing_size'] = df_listified['item'].apply(lambda x: get_packing_size(x))
df_listified['num_units'] = df_listified['item'].apply(lambda x: dict_listified[x])
df_listified['weight'] = df_listified.packing_size * df_listified.num_units
df_listified['unit'] = df_listified['item'].apply(lambda x: get_unit(x))

In [37]:
df_listified = df_listified[df_listified.num_units > 0]; df_listified

,item,packing_size,num_units,weight,unit
0,Potatoes,1.00,66.0,66.00,Kg
1,Red Onions,1.00,69.0,69.00,Kg
2,Cauliflower,1.00,48.0,48.00,Pieces
3,Cabbage,1.00,23.0,23.00,Pieces
4,Brinjal Bharta,0.50,20.0,10.00,Kg
5,Pumpkin,0.50,31.0,15.50,Kg
6,Brinjal Kateri,0.50,14.0,7.00,Kg
7,Cucumber,0.50,55.0,27.50,Kg
8,Bhindi/Lady's Finger,0.50,45.0,22.50,Kg
9,Bitter Gourd,0.30,16.0,4.80,Kg


In [38]:
fname = save_pth/('quantity-' + str(FILE.stem) + '.csv')

In [39]:
df_listified[df_listified.num_units>0].to_csv(fname, index=False)

# Customerwise

In [40]:
items = ['coconut', 'kiwi']

In [41]:
df_trans = pd.read_excel("C:/Playground/bhaiya-orders/zoho-bhor-farms/database/translations.xlsx")

In [42]:
from fuzzywuzzy import fuzz

def get_match(x, thresh=65):
    x = x.lower()
    rmax = -1
    for k in items: 
        if fuzz.partial_ratio(k, x) > rmax:
            rmax = fuzz.partial_ratio(k, x)

    return rmax > thresh

In [43]:
keys2drop = df_trans[df_trans['Final Item Name'].apply(get_match)]['Final Item Name'].values; keys2drop

array(['Coconut', 'Kiwi Imported', 'Coconut Mangalore Small',
       'Kiwi Imported', ' Coconut Mangalore Big',
       ' Coconut Mangalore Medium', 'Kiwi Imported - Zespri Green'],
      dtype=object)

In [44]:
df_trans = df_trans.set_index('Final Item Name').drop(keys2drop).reset_index()
eng2marathi = df_trans.set_index('Final Item Name').to_dict()['Final Marathi Name']
marathi2eng = df_trans.set_index('Final Marathi Name').to_dict()['Final Item Name']
marathi2eng[np.nan] = ''

In [45]:
def get_trans(x, thresh=85):
    rmax = -1
    trans = ''
    for k in eng2marathi.keys():
        if fuzz.ratio(k, x) > rmax:
            rmax = fuzz.ratio(k, x)
            trans = eng2marathi[k]

    if rmax > thresh:
        return trans
    return np.nan

In [46]:
rows_list = []
for token, o in orders_dict.items():
    for item, qty in o['order'].items():
        row_dct = {}
        row_dct['token_no'] = token
        row_dct['name'] = o['name']
        row_dct['delivery_date'] = o['delivery_date']
        row_dct['item_name'] = item
        row_dct['marathi_name'] = get_trans(item)
        row_dct['num_packets'] = qty
        row_dct['packing_size'] = get_packing_size(item)
        row_dct['final_weight'] = row_dct['num_packets'] * row_dct['packing_size']
        row_dct['unit'] = get_unit(item)
        row_dct['total_amt'] = o['total_amount']
        
        rows_list.append(row_dct)

In [47]:
df_customerwise = pd.DataFrame(rows_list)

In [48]:
df_customerwise.head()

,token_no,name,delivery_date,item_name,marathi_name,num_packets,packing_size,final_weight,unit,total_amt
0,1,Kumkum; Jain,Thu Oct 29th,Potatoes,बटाटा,2.0,1.0,2.0,Kg,791
1,1,Kumkum; Jain,Thu Oct 29th,Cauliflower,फ्लॉवर,1.0,1.0,1.0,Pieces,791
2,1,Kumkum; Jain,Thu Oct 29th,Cabbage,कोबी,1.0,1.0,1.0,Pieces,791
3,1,Kumkum; Jain,Thu Oct 29th,Brinjal Bharta,भरता वांगे,1.0,0.5,0.5,Kg,791
4,1,Kumkum; Jain,Thu Oct 29th,Cucumber,काकडी,2.0,0.5,1.0,Kg,791


In [49]:
notranslist = df_customerwise[df_customerwise.marathi_name.isna()].item_name.unique(); notranslist

array(['Kiwi Zespri Imported', 'Indian Avacado'], dtype=object)

## Filter Out Exotics

In [50]:
notranslist = ['Button Mushroom',
    'Broccoli',
    'Lettuce',
    'Kiwi Zespri Imported',
    'Babycorn',
    'Peeled Garlic',
    'Green Zucchini',
    'Yellow Zucchini',
    'Pumpkin',
    'Red/Yellow Bell Peppers',
    'Indian Avacado',
    ]

In [51]:
ids = set([i.lower()[:2] for i in notranslist])
prefix = "".join(ids)
prefix = prefix.replace('/', '')

notrans_pth = save_pth/f'{prefix}-{str(FILE.stem)}.csv'; notrans_pth.stem

'grkilereinbuyepepubrba-Final Input 29th Veg'

In [52]:
df_customerwise[df_customerwise.item_name.isin(notranslist)].to_csv(notrans_pth, index=False)

In [53]:
# (df_customerwise[df_customerwise.marathi_name.isna()]
#     .drop(columns=['marathi_name', 'total_amt'])
#     .to_csv(notrans_pth, index=False))

## Drop items whose translations are absent from the Customerwise File

In [54]:
df_customerwise.marathi_name = df_customerwise.marathi_name.fillna('')

In [55]:
# df_customerwise.dropna(subset=['marathi_name'], inplace=True)

In [56]:
df_customerwise.to_csv(save_pth/'full-customerwise.csv', index=False)

## Build Excel Savefile

In [57]:
highlight_col = 'num_packets'

In [58]:
custs = df_customerwise.name + '_' + df_customerwise.token_no.astype(str)
custs = custs.unique()

In [59]:
def f(x):
    col = highlight_col
    r = 'background-color: none'
    g = 'background-color: #37FDFC'
    c = np.where(x[col] > 1, g, r)
    y = pd.DataFrame('', index=x.index, columns=x.columns)
    for h in all_cols:
        y[h] = c
    
    return y

In [60]:
def boldify(x):
    col = 'total_amt'
    n = 'font-weight: normal'
    b = 'font-weight: bold'
    c = np.where(x[col] >= 1000.0, b, n)
    y = pd.DataFrame('', index=x.index, columns=x.columns)
    for h in all_cols:
        y[h] = c
    
    return y

In [61]:
all_cols = df_customerwise.columns

In [62]:
def get_sum_packets(token):
    return sum(orders_dict[token]['order'].values())

In [63]:
fname = save_pth/('customerwise-' + str(FILE.stem) + '.xlsx')

In [64]:
all_data = 0

with pd.ExcelWriter(fname) as writer:
    for cust in custs:
        data = df_customerwise[df_customerwise.name == cust.split('_')[0]]
        data = data[data.token_no == int(cust.split('_')[1])]
        all_data += len(data.index)

        data = data.append({'marathi_name': 'TOTAL PACKETS', 'num_packets': get_sum_packets(
            int(cust.split('_')[1]))}, ignore_index=True)
        styled = data.style.apply(f, axis=None)
        styled = styled.apply(boldify, axis=None)
        styled.to_excel(writer, columns=['token_no', 'name', 'delivery_date', 'item_name', 'marathi_name',
                                         'num_packets', 'packing_size', 'final_weight', 'unit'], sheet_name=cust[:31], engine='openpyxl', index=False)

assert all_data == len(df_customerwise)

## Build PDF Savefile

In [65]:
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfbase.ttfonts import TTFont  
from reportlab.pdfbase import pdfmetrics
from reportlab.platypus import Table, TableStyle, PageBreak, SimpleDocTemplate
from reportlab.lib import colors

pdfmetrics.registerFont(TTFont("mangal", "font/Lohit-Devanagari.ttf"))

In [66]:
df_customerwise.drop(columns=['delivery_date', 'total_amt'], inplace=True)

In [67]:
# get index of num_packets, marathi_name
for i, col in enumerate(df_customerwise.columns): 
    if col == highlight_col: qty_index = i
    if col == 'marathi_name': marathi_index = i

In [68]:
def create_pdf(data):
    
    pdfy_data = data.to_numpy().tolist()
    pdfy_data.insert(0, data.columns.values.tolist())

#     pdf = SimpleDocTemplate(
#         fname,
#         pagesize=landscape(letter),
#     #     font='mangal'
#     )

    table = Table(pdfy_data)

    style = TableStyle([
    #     ('BACKGROUND', (0,0), (3,0), colors.green),
    #     ('TEXTCOLOR',(0,0),(-1,0),colors.whitesmoke),

        ('ALIGN',(0,0),(-1,-1),'CENTER'),
    #     ('FONTNAME', (0,0), (-1,0), 'Courier-Bold'),
        ('FONTSIZE', (0,0), (-1,0), 10),
        ('BOTTOMPADDING', (0,0), (-1,0), 12),
        ('FONTNAME',(marathi_index,1),(marathi_index,-1),'mangal'),
        ('BACKGROUND',(0,0),(-1,0),colors.burlywood),
        ('BACKGROUND',(qty_index,-1),(qty_index,-1),colors.springgreen)
    ])
    
    table.setStyle(style)

    rowNumb = len(pdfy_data)
    for i in range(1, rowNumb-1):
        if pdfy_data[i][qty_index] > 1:
            bc = colors.aqua
        else:
            bc = colors.white
        ts = TableStyle(
            [('BACKGROUND', (0,i),(-1,i), bc)]
        )
        table.setStyle(ts)

    # 3) Add borders
    ts = TableStyle(
        [
    #     ('BOX',(0,0),(-1,-1),2,colors.black),

    #     ('LINEBEFORE',(2,1),(2,-1),2,colors.red),
    #     ('LINEABOVE',(0,2),(-1,2),2,colors.green),

        ('GRID',(0,0),(-1,-2),2,colors.black),
        ]
    )
    table.setStyle(ts)

#     elems = []
    elems.append(table)
    elems.append(PageBreak())
    
#     pdf.build(elems)

In [69]:
fname = str(PDF_DIR/('customerwise-' + str(FILE.stem) + '.pdf'))

In [70]:
pdf = SimpleDocTemplate(
        fname,
        pagesize=landscape(letter),
    #     font='mangal'
    )

elems = []

In [71]:
all_data = 0

for cust in custs:
    data = df_customerwise[df_customerwise.name == cust.split('_')[0]]
    data = data[data.token_no == int(cust.split('_')[1])]
    
    data.token_no = data.token_no.astype(int)
    data.num_packets = data.num_packets.astype(int)
    
    all_data += len(data.index)
    
    data = data.append({'num_packets': get_sum_packets(
        int(cust.split('_')[1]))}, ignore_index=True)

    data = data.fillna('')
    create_pdf(data)

assert all_data == len(df_customerwise)

In [72]:
pdf.build(elems)

# Printout File

In [108]:
# decision for vegetables vs seasonal fruits

In [73]:
def is_trans(x, thresh=85):
    rmax = -1
    trans = ''
    for k in eng2marathi.keys():
        if fuzz.ratio(k, x) > rmax:
            rmax = fuzz.ratio(k, x)
            trans = eng2marathi[k]

    return rmax > thresh

In [74]:
translations = [is_trans(i) for i in all_items]

In [75]:
pipe_separated = sum(translations)/len(translations) < 0.4 # less than 40% translations are present

In [76]:
pipe_separated

False

## Sudeep

In [77]:
def get_pipe_separated(token):
    order = orders_dict[token]['order']
    s=[]
    for k, v in order.items():
        s.append(k + ': ' + str(int(v)))
    
    return ' | '.join(s)

In [78]:
rows_list_sudeep = []
for token, o in orders_dict.items():
    row_dct = {}
    row_dct['token_no'] = token
    row_dct['name'] = o['name']
    row_dct['drop_point'] = o['drop_point']
    row_dct['building'] = o['building']
    row_dct['wing'] = o['wing']
    row_dct['flat_no'] = o['flat_no']
    row_dct['phone_no'] = o['phone_no']
    row_dct['email'] = o['email']
    
    if pipe_separated: row_dct['items'] = get_pipe_separated(token)
    else: row_dct['num_packets'] = get_sum_packets(token)
    
    row_dct['total_amount'] = o['total_amount']
    row_dct['total_cp'] = o['total_cost_price']
    row_dct['profit'] = o['profit']
    
    
    rows_list_sudeep.append(row_dct)

In [79]:
df_print_sudeep = pd.DataFrame(rows_list_sudeep)

In [80]:
df_print_sudeep.head()

,token_no,name,drop_point,building,wing,flat_no,phone_no,email,num_packets,total_amount,total_cp,profit
0,1,Kumkum; Jain,Ajmera iLand,Aeon,A Wing,302,9930160637,Jainkmkm@gmail.com,26.0,791,26.0,765.0
1,2,Rashmita; Sundaram,Ajmera iLand,Aeon,A Wing,1202,9004913659,Rashmita.sundaram@gmail.com,11.0,412,11.0,401.0
2,3,Neha; Aggarwal,Ajmera iLand,Aeon,A Wing,101,9999488247,Naggdce@gmail.com,25.0,877,25.0,852.0
3,4,Ruchi; Mundhra,Ajmera iLand,Treon,A Wing,1603,919920748892,mundhra.ruchi72@gmail.com,8.0,266,8.0,258.0
4,5,Prerna; Jain,Ajmera iLand,Zeon,A Wing,2403,9920579019,prerna206@gmail.com,39.0,1567,39.0,1528.0


In [81]:
save_pth

WindowsPath('C:/Playground/bhaiya-orders/zoho-bhor-farms/outputs/Final Input 29th Veg')

In [82]:
df_print_sudeep.to_csv(save_pth/'print-sudeep.csv', index=False)

## Vishal

In [83]:
rows_list_vishal = []
for token, o in orders_dict.items():
    row_dct = {}
    row_dct['token_no'] = token
    row_dct['name'] = o['name']
    row_dct['drop_point'] = o['drop_point']
    row_dct['building'] = o['building']
    row_dct['wing'] = o['wing']
    row_dct['flat_no'] = o['flat_no']
#     row_dct['phone_no'] = o['phone_no']
#     row_dct['email'] = o['email']

    if pipe_separated: row_dct['items'] = get_pipe_separated(token)
    else: row_dct['num_packets'] = get_sum_packets(token)    
    
    if 'total_amount' in o.keys():
        row_dct['total_amount'] = o['total_amount']
    
    rows_list_vishal.append(row_dct)

In [84]:
df_print_vishal = pd.DataFrame(rows_list_vishal)

In [85]:
df_print_vishal.head()

,token_no,name,drop_point,building,wing,flat_no,num_packets,total_amount
0,1,Kumkum; Jain,Ajmera iLand,Aeon,A Wing,302,26.0,791
1,2,Rashmita; Sundaram,Ajmera iLand,Aeon,A Wing,1202,11.0,412
2,3,Neha; Aggarwal,Ajmera iLand,Aeon,A Wing,101,25.0,877
3,4,Ruchi; Mundhra,Ajmera iLand,Treon,A Wing,1603,8.0,266
4,5,Prerna; Jain,Ajmera iLand,Zeon,A Wing,2403,39.0,1567


In [86]:
df_print_vishal.to_csv(save_pth/'print-vishal.csv', index=False)

## Dropwise 

In [87]:
df_print_vishal.drop_point = df_print_vishal.drop_point.fillna('')

In [88]:
drops = df_print_vishal.drop_point.unique()

In [89]:
# fname = save_pth/"dropwise-phool.xlsx"
fname = save_pth/('dropwise-' + str(FILE.stem) + '.xlsx')

In [90]:
drops

array(['Ajmera iLand', 'Cosmic', 'Dosti Acres', 'Ivory', 'Lodha NCP',
       'R1', 'R12', 'R2', 'Sector 1/2'], dtype=object)

In [91]:
df_print_sudeep.loc[df_print_sudeep.drop_point == 0]

,token_no,name,drop_point,building,wing,flat_no,phone_no,email,num_packets,total_amount,total_cp,profit


In [92]:
df_print_sudeep.loc[df_print_sudeep.drop_point == 0, 'drop_point'] = ['']

In [93]:
all_data = 0

with pd.ExcelWriter(fname) as writer:
    for drop in drops:
        data = df_print_vishal[df_print_vishal.drop_point == drop]
        all_data += len(data.index)

        invalid_chars = '[]:*?/\\'
        for c in invalid_chars: drop = drop.replace(c, '')
        
        data.to_excel(writer, sheet_name=drop, index=False)
    
assert all_data == len(df_print_vishal)

## Tokens

In [94]:
subset_cols = ['name', 'token_no', 'building', 'wing', 'flat_no']

In [95]:
df_tokens = df_print_vishal[subset_cols]

In [96]:
drops = df_print_vishal.drop_point.unique()

In [97]:
# fname = save_pth/"tokens-phool.xlsx"
fname = save_pth/('tokens-' + str(FILE.stem) + '.xlsx')

In [98]:
all_data = 0

with pd.ExcelWriter(fname) as writer:
    for drop in drops:
        data = df_print_vishal[df_print_vishal.drop_point == drop]
        all_data += len(data.index)
        
        invalid_chars = '[]:*?/\\'
        for c in invalid_chars: drop = drop.replace(c, '')
        
        data.to_excel(writer, sheet_name=drop, columns=subset_cols, index=False)
    
assert all_data == len(df_print_vishal)

# Dropwise Count

In [99]:
rows_list = []
for token, order in orders_dict.items():
    for item, count in order['order'].items():
        row_dct = {}
        row_dct['drop_point'] = order['drop_point']
        row_dct['item_name'] = item
        row_dct['num_packets'] = count
        
        rows_list.append(row_dct)

In [100]:
df_dropwise = pd.DataFrame(rows_list)

In [101]:
df_dropwise

,drop_point,item_name,num_packets
0,Ajmera iLand,Potatoes,2.0
1,Ajmera iLand,Cauliflower,1.0
2,Ajmera iLand,Cabbage,1.0
3,Ajmera iLand,Brinjal Bharta,1.0
4,Ajmera iLand,Cucumber,2.0
...,...,...,...
985,Sector 1/2,Lemons,1.0
986,Sector 1/2,Curry Leaves,1.0
987,Sector 1/2,Coriander Leaves,1.0
988,Sector 1/2,Carrots,1.0


In [102]:
fname = save_pth/('dropwisecount-' + str(FILE.stem) + '.csv')

In [103]:
df_dropwise.groupby(['drop_point', 'item_name']).sum().reset_index().to_csv(fname, index=False)

In [104]:
!explorer C:/Playground/bhaiya-orders/zoho-bhor-farms/outputs/

# Send Message Alerts

In [105]:
import http.client
conn = http.client.HTTPSConnection("api.msg91.com")

In [106]:
df.Phone = df.Phone.astype(str).apply(lambda x: x[-10:])

In [107]:
headers = {
    'authkey': "329435AXayMy78QG5f914544P1",
    'content-type': "application/json"
    }

In [109]:
pipe_separated

False

In [112]:
def get_message(first_name, token, amount, mobile, payment_status, date, decision):
    if decision: product = 'Fruits/Dry Fruits'
    else: product = 'Vegetables'
    
    unsuccessful_message = f"Hi {first_name},\nYour {product} order with Token no {int(token)} from Bhor Farms is confirmed and will be delivered to your lobby by the evening of {date}.\nYour bill amount is Rs {amount}. Please check your SMS/Email for payment link.\nCall 6203507070 for any queries."
    successful_message = f"Hi {first_name},\nYour {product} order with Token no {int(token)} from Bhor Farms is confirmed and will be delivered to your lobby by the evening of {date}.\nCall 6203507070 for any queries."
    
    if payment_status == 'Completed':
        msg_json = { "sender": "BHRFMS", 
                     "route": "4", 
                     "country": "91", 
                     "sms": [ 
                         { "message": successful_message, 
                          "to": [f"{mobile}"] } ] }
        
    else:
        msg_json = { "sender": "BHRFMS", 
                     "route": "4", 
                     "country": "91", 
                     "sms": [ 
                         { "message": unsuccessful_message, 
                          "to": [f"{mobile}"] } ] }
        
    return str(msg_json).replace('\'', '\"')

In [113]:
responses = {}

for index, row in df.iterrows():
    message = get_message(row['Name'].split(';')[0].capitalize(), 
                          row['Token No'], 
                          row['Total Amount'], 
                          row['Phone'], 
                          row['Payment Status'],
                          row['Select Delivery Date'],
                          pipe_separated
                         )
    
    conn.request("POST", "/api/v2/sendsms", message, headers)
    res = conn.getresponse()
    data = res.read()
    responses[row['Phone']] = data.decode("utf-8")

In [114]:
responses

{'9930160637': '{"message":"5f984ae5d6fc053017177616","type":"success"}',
 '9004913659': '{"message":"5f984ae5d6fc052ecc3c4c6b","type":"success"}',
 '9999488247': '{"message":"5f984ae6d6fc0556233e4222","type":"success"}',
 '9920748892': '{"message":"5f984ae6d6fc05113671986b","type":"success"}',
 '9920579019': '{"message":"5f984ae6d6fc057f4f7f4732","type":"success"}',
 '9820157581': '{"message":"5f984ae6d6fc05461d1a1e58","type":"success"}',
 '9422212171': '{"message":"5f984ae6d6fc05588b7cb5c4","type":"success"}',
 '9867041235': '{"message":"5f984ae6d6fc053293578be4","type":"success"}',
 '9769656431': '{"message":"5f984ae7d6fc053bc1508b6f","type":"success"}',
 '9930014369': '{"message":"5f984ae7d6fc05112d357ac9","type":"success"}',
 '9930851066': '{"message":"5f984ae7d6fc054405514045","type":"success"}',
 '9599219896': '{"message":"5f984ae7d6fc055e1f6c5b51","type":"success"}',
 '9821026234': '{"message":"5f984ae7d6fc0518b45cb23b","type":"success"}',
 '9967716023': '{"message":"5f984ae8d6

In [115]:
message

'{"sender": "BHRFMS", "route": "4", "country": "91", "sms": [{"message": "Hi Sheeba,\\nYour Vegetables order with Token no 79 from Bhor Farms is confirmed and will be delivered to your lobby by the evening of Thu Oct 29th.\\nCall 6203507070 for any queries.", "to": ["9821901596"]}]}'

# End

## Manually Adding Orders

In [125]:
for a in all_items: print(a)

Apples New Zealand
Indian Pear
Imported Green Pear
VNR Guava
Mosambi
Sitaphal
Assorted Mango Box
Muskmelon
Elaichi Banana Regular
Elaichi Banana Premium
Kiwi Zespri
Pomegranate Premium
Pomegranate Regular
Langra Mangoes
Chausa Mangoes
Papaya
Watermelon
Amla
Haldi/Turmeric Powder
Premium Long Raisins
Coconut Mangalore Small
Coconut Mangalore Medium
Coconut Mangalore Large
Dragon Fruit White Flesh
Himachal Apples
Imported Oranges
Imported Plum
Pure Kashmiri Sidr Honey
Alphonso Mango Pulp Frozen
Premium Imported Apricots
Cashew W320
Cashew W240
Cashew W210
Peri Peri Cashew
Broken Cashew
Standard Walnuts
Premium Walnuts
Salted Pista Standard
Salted Pista Premium
Anjeer Premium
Anjeer Super Premium
Mix Dry Fruits
Mamra Badam Medium
Kishmish Round
Kishmish Long
Munakka Indian
Imported Munakka Afgan
Jardalu/Dried Apricots
Kimia Dates
Kashmiri Qahwah Tea
Litchi Juice
7 Fruits Juice
Guava Juice
Kharbooja Juice
Strawberry Juice
Pineapple Juice
Anaar Juice
Orange Juice
Kiwi Juice


In [57]:
new_orders = pd.read_csv('yelofile/additions.csv')

In [58]:
keys1 = ['name', 'drop_point', 'building', 'wing', 'flat_no', 'phone_no', 'email', 'delivery_date', 'total_amount', 'order']

In [59]:
new_orders.dropna(inplace=True)

In [60]:
new_orders

,index,Haar 1,Haar 2,Jhendu/Marigold,Rajnigandha,Druva,Tulsi,Pack of 5 leaves,Betelnut,Banana Leaves with Stem,Banana Leaves,Pack of 5 Fruits,Assorted Flowers,Hibiscus


In [133]:
to_append = new_orders.to_dict(orient='records')

In [134]:
orders_very_very_new = pd.read_csv('yelofile/additions.csv')
orders_very_very_new.set_index('index', inplace=True)
orders_very_very_new.fillna(0, inplace=True)

In [137]:
for i in to_append:
    i['order'] = new_items[i['name']]

In [61]:
to_append = [{'name': 'Suchitra;Narale',
  'drop_point': 'Cosmic',
  'building': 'Cosmic Heights',
  'wing': 'B Wing',
  'flat_no': 2603.0,
  'phone_no': 9833567340.0,
  'email': 'suchitrahate@gmail.com',
  'order': {'Haar 1': 1.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 1.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 0.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 0.0,
   'Pack of 5 Fruits': 1.0,
   'Assorted Flowers': 1,
   'Hibiscus': 0.0}},
 {'name': 'Ritu;Sharda',
  'drop_point': 'Cosmic',
  'building': 'Cosmic Heights',
  'wing': 'B Wing',
  'flat_no': 2101.0,
  'phone_no': 9820844319.0,
  'email': 'shardaritu@hotmail.com',
  'order': {'Haar 1': 0.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 2.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 1.0,
   'Pack of 5 Fruits': 0.0,
   'Assorted Flowers': 1,
   'Hibiscus': 0.0}},
 {'name': 'Nisha;Narale',
  'drop_point': 'Cosmic',
  'building': 'Cosmic Heights',
  'wing': 'A Wing',
  'flat_no': 2505.0,
  'phone_no': 9930390994.0,
  'email': 'nishashetti@gmail.com',
  'order': {'Haar 1': 1.0,
   'Haar 2': 1.0,
   'Jhendu/Marigold': 0.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 1.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 1.0,
   'Pack of 5 Fruits': 2.0,
   'Assorted Flowers': 3,
   'Hibiscus': 1.0}},
 {'name': 'Padmaja;Kakkera',
  'drop_point': 'Lodha NCP',
  'building': 'Tower 4',
  'wing': 'B Wing',
  'flat_no': 202.0,
  'phone_no': 9900065600.0,
  'email': 'padmaja.kakkera@gmail.com',
  'order': {'Haar 1': 0.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 0.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 1.0,
   'Pack of 5 leaves': 1.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 0.0,
   'Pack of 5 Fruits': 1.0,
   'Assorted Flowers': 1,
   'Hibiscus': 1.0}},
 {'name': 'Shalini;Memani',
  'drop_point': 'Lodha NCP',
  'building': 'Tower 6',
  'wing': 'A Wing',
  'flat_no': 2503.0,
  'phone_no': 9323134454.0,
  'email': 'shalini_depura@yahoo.co.in',
  'order': {'Haar 1': 2.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 0.0,
   'Rajnigandha': 0.0,
   'Druva': 2.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 2.0,
   'Pack of 5 Fruits': 0.0,
   'Assorted Flowers': 1,
   'Hibiscus': 3.0}},
 {'name': 'Ila Panchakshari',
  'drop_point': 'Ivory',
  'building': 'Himalayan Heights',
  'wing': 'A Wing',
  'flat_no': 1003.0,
  'phone_no': 8286125928.0,
  'email': 'ilahp78@gmail.com',
  'order': {'Haar 1': 0.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 0.0,
   'Rajnigandha': 0.0,
   'Druva': 5.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 0.0,
   'Pack of 5 Fruits': 0.0,
   'Assorted Flowers': 1,
   'Hibiscus': 1.0}},
 {'name': 'Swati;Madhukar',
  'drop_point': 'Lodha NCP',
  'building': 'Tower 6',
  'wing': 'A Wing',
  'flat_no': 4002.0,
  'phone_no': 9167853838.0,
  'email': 'swati.chaudhary90@yahoo.com',
  'order': {'Haar 1': 8.0,
   'Haar 2': 2.0,
   'Jhendu/Marigold': 0.0,
   'Rajnigandha': 0.0,
   'Druva': 0.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 5.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 0.0,
   'Pack of 5 Fruits': 0.0,
   'Assorted Flowers': 1,
   'Hibiscus': 0.0}}]

In [68]:
new_orders_dict = {}
i = 178

for row in to_append:
    dct = {}
    dct['name'] = row['name']
#     dct['drop_point'] = row['Select Drop Point']
#     dct['building'] = row['building']
#     dct['wing'] = row['Select Wing']
#     dct['flat_no'] = row['Flat No']
    dct['phone_no'] = row['phone_no']
    dct['email'] = row['email']
#     dct['delivery_date'] = row['Select Delivery Date']
#     dct['total_amount'] = row['Total Amount']
#     dct['total_amount'] = 
#     dct['payment_status'] = row['Payment Status']
    dct['order'] = get_order(row)
    dct['total_amount'] = get_total_amt(dct['order'])
    
    new_orders_dict[i] = dct
    i+=1

In [70]:
60+100+15+130+121

426

In [71]:
480+200+25+50+121

876

In [73]:
120+30+25+121

296

In [80]:
items_dict

{'Haar 1': {'packing_size': 1, 'unit': 1, 'sell_price': 60},
 'Haar 2': {'packing_size': 1, 'unit': 1, 'sell_price': 100},
 'Jhendu/Marigold': {'packing_size': 1, 'unit': 1, 'sell_price': 100},
 'Druva': {'packing_size': 1, 'unit': 1, 'sell_price': 15},
 'Tulsi': {'packing_size': 1, 'unit': 1, 'sell_price': 20},
 'Pack of 5 leaves': {'packing_size': 1, 'unit': 1, 'sell_price': 25},
 'Betelnut': {'packing_size': 1, 'unit': 1, 'sell_price': 25},
 'Banana Leaves with Stem': {'packing_size': 1, 'unit': 1, 'sell_price': 100},
 'Pack of 5 Fruits': {'packing_size': 1, 'unit': 1, 'sell_price': 130},
 'Assorted Flowers': {'packing_size': 1, 'unit': 1, 'sell_price': 121},
 'Hibiscus': {'packing_size': 1, 'unit': 1, 'sell_price': 20}}

In [138]:
to_append

[{'name': 'Suchitra;Narale',
  'drop_point': 'Cosmic',
  'building': 'Cosmic Heights',
  'wing': 'B Wing',
  'flat_no': 2603.0,
  'phone_no': 9833567340.0,
  'email': 'suchitrahate@gmail.com',
  'order': {'Haar 1': 1.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 1.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 0.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 0.0,
   'Pack of 5 Fruits': 1.0,
   'Assorted Flowers': 1,
   'Hibiscus': 0.0}},
 {'name': 'Ritu;Sharda',
  'drop_point': 'Cosmic',
  'building': 'Cosmic Heights',
  'wing': 'B Wing',
  'flat_no': 2101.0,
  'phone_no': 9820844319.0,
  'email': 'shardaritu@hotmail.com',
  'order': {'Haar 1': 0.0,
   'Haar 2': 0.0,
   'Jhendu/Marigold': 2.0,
   'Rajnigandha': 0.0,
   'Druva': 1.0,
   'Tulsi': 0.0,
   'Pack of 5 leaves': 0.0,
   'Betelnut': 1.0,
   'Banana Leaves with Stem': 0.0,
   'Banana Leaves': 1.0,
   'Pack of 5 Fruits': 0.0,
   'Assorted Flowers': 1,
   'Hibiscus':

In [ ]:
for k in to_a

In [136]:
new_items = orders_very_very_new.to_dict(orient='index')

In [74]:
keys2 = ['Haar 2', 'Druva', 'Tulsi', 'Pack of 5 leaves', 'Betelnut', 'Banana Leaves with Stem', 'Banana Leaves', 'Pack of 5 Fruits', 'Assorted Flowers']

In [86]:
for k in keys2:
    print (k)

Haar 2
Druva
Tulsi
Pack of 5 leaves
Betelnut
Banana Leaves with Stem
Banana Leaves
Pack of 5 Fruits
Assorted Flowers


In [66]:
df_dropwise.groupby(['drop_point', 'item_name']).sum()

num_packets
drop_point   item_name                           
Ajmera iLand Assorted Flowers                12.0
             Banana Leaves                    6.0
             Banana Leaves with Stem          6.0
             Betelnut                         7.0
             Druva                           20.0
...                                           ...
Sector 1/2   Jhendu/Marigold                  7.0
             Pack of 5 Fruits                10.0
             Pack of 5 leaves                 6.0
             Rajnigandha                      1.0
             Tulsi                            4.0

[62 rows x 1 columns]

In [62]:
for token in range(178, 178+len(to_append)):
    orders_dict[token] = to_append[178-token]

## Pipe Separated Tatkaal

In [4]:
file = Path(fc.selected)

In [6]:
df = pd.read_excel(file)

In [14]:
# CHANGE MANUALLY EVERYTIME
allcols = df.columns[9:-2]

In [15]:
df.fillna('', inplace=True)

In [16]:
df['piped'] = ''

In [18]:
piped = []
for _, row in df.iterrows():
    tmp = []
#     print(row[df.columns[11:-2]])
#     break
    for col in allcols: 
        if(row[col] != ''):
            tmp.append(row[col])
#             tmp.append(col + ' ' + str(int(row[col])))
    
    piped.append(" | ".join(tmp))

In [19]:
df.piped = piped

In [20]:
df.head()

,Submission Date,Order Number#,First Name,Last Name,Select Drop Point and then Building,Select Wing,Flat No,Phone Number,Email,Cucumber,Lemon,Capsicum,Ginger,Lady's Finger/Bhindi,Garlic,Baingan Bharta,Carrots,French Beans,Tomato,Bottle Gourd,Bitter Gourd,Cabbage,Mushrooms,Cauliflower,Lal Math/Amaranthus Leaves,Sweet Corn,Lemongrass,Sponge Gourd,Coriander,Shepu/Dill Leaves,Pudina/Mint Leaves,Palak,Broccoli,Premium Sitaphal,Total Amount,Any pending unadjusted credit from previous orders?,piped
0,2020-09-13 11:04:37,BHORD-0165,Gunjan,Churiwal,Ivory - Julian Alps,B Wing,403,9821902333,higunjan2002@gmail.com,,,,,,,,,,,,,,,,,,,,,,,,,Premium Sitaphal : 1,150,,Premium Sitaphal : 1
1,2020-09-13 11:03:22,BHORD-0164,Priyanka,Chordia,NCP - Tower 4,B Wing,3603,9819984779,priyanka.chordia2886@gmail.com,,,,,,Garlic : 1,,,,Tomato : 1,,,,,,,,,,,,,,,,95,,Garlic : 1 | Tomato : 1
2,2020-09-13 11:02:20,BHORD-0163,Sonal,Akkara,NCP - Tower 5,B Wing,4002,9560374999,sosh982@yahoo.com,,,,,,,,,,,,,,,,,,,,,,,,,Premium Sitaphal : 1,150,,Premium Sitaphal : 1
3,2020-09-13 10:56:37,BHORD-0162,Amlan,Chaudhuri,NCP - Tower 6,B Wing,2804,9820475334,chaudhuri.amlan@gmail.com,,,,,,,,,,,,,,,,,,,,,,,,,Premium Sitaphal : 1,150,,Premium Sitaphal : 1
4,2020-09-13 10:55:44,BHORD-0161,Praveen,Sangal,NCP - Tower 7,B Wing,1504,9820974386,praveensangal@gmail.com,,,,,,,,,,,,,,,,,,,,,,,,,Premium Sitaphal : 1,150,,Premium Sitaphal : 1


In [38]:
df.to_csv('piped-tatkaal.csv', index=False)